# Making predictions using streaming aggregated features

All prior notebooks have been setting up the solution. Now that all those steps are complete, it is time to see the solution in action. In this notebook, we will send credit card transactions to our input stream and show that we can detect fraud. We take advantage of multiple online feature groups in the Amazon SageMaker Feature Store. One of those groups is refreshed by a processing job, which we would run nightly to provide aggregate features looking back one week. The other feature group uses streaming ingestion to aggregate features that look back over a rolling 10-minute window.

### Recap of what is in place

Here is a recap of what we have done so far:

1. We generated a synthetic dataset of transactions, including simulated fraud attacks.
2. We created a pair of feature groups.
3. We created a Kinesis data stream and a Kinesis Data Analytics SQL application that consumes the transaction stream and produces aggregate features. These features are provided in near real time to Lambda, and they look back over a 10 minute window.
4. We created aggregated features and used them to feed both the training dataset as well as an online feature group.
5. We trained and deployed an XGBoost model to detect fraud.
6. Our CloudFormation template deployed a pair of Lambda functions. One listens to the KDA SQL output and keeps the `cc-agg-fg` feature group up to date.
7. The other Lambda listens to the Kinesis data stream for transactions, pulls a set of features from multiple feature groups, and invokes our fraud detection endpoint.

## Imports and overall setup

### Imports and initialization

In [1]:
from datetime import datetime
import numpy as np
import pandas as pd
import sagemaker
import boto3
import json
import time

In [141]:
LOCAL_DIR = './data'
BUCKET = sagemaker.Session().default_bucket()
PREFIX = 'testing'
STREAM_NAME = 'cc-stream'

s3_client = boto3.Session().client('s3')
kinesis_client = boto3.client('kinesis')

In [138]:
# The exact name of the Lambda function is controlled by our CloudFormation template, 
# so we access that here. We will use this to help get to the proper CloudWatch log group to see the
# results of our testing.
%store -r
predict_lambda_name

'InvokeFraudEndpointLambda-v4'

### Ensure Lambda knows which SageMaker endpoint to use
In our previous notebook that deploys a SageMaker endpoint, we allow the endpoint name to be generated on the fly instead of hard-coding a specific endpoint name. Our Lambda function that invokes the endpoint thus needs a way to know the endpoint name. We handle that through a Lambda environment variable.

This section of code simply takes care of updating end ENDPOINT_NAME Lambda environment variable. It is important to do so before we start feeding transactions into our Kinesis stream.

In [3]:
lambda_client = boto3.client('lambda')

# Grab the latest endpoint name we used in the previous notebook, as well as the ARN for the lambda
%store -r
print(f'Updating Lambda to use endpoint: {endpoint_name} for ARN: {lambda_to_model_arn}')

variables = lambda_client.get_function_configuration(FunctionName=lambda_to_model_arn)['Environment']['Variables']
variables['ENDPOINT_NAME'] = endpoint_name
resp = lambda_client.update_function_configuration(
    FunctionName=lambda_to_model_arn,
      Environment={
        'Variables': variables
    }
)

Updating Lambda to use endpoint: sagemaker-xgboost-2020-12-07-23-25-29-104 for ARN: arn:aws:lambda:us-east-2:105242341581:function:InvokeFraudEndpointLambda-v4


### Access the transaction test dataset

In [4]:
test_file_path = './data/test.csv'
test_df = pd.read_csv(test_file_path)
test_df.head()

,tid,datetime,cc_num,amount,fraud_label,num_trans_last_10m,avg_amt_last_10m,num_trans_last_1w,avg_amt_last_1w,amt_ratio1,amt_ratio2,count_ratio
0,b66d85a66671654ee1ce894f472ad0ed,2020-05-15T02:15:48.000Z,4001487383951324,15.61,0,1,15.61,26,1111.710769,0.014041,0.014041,0.038462
1,ea24ea401ee3e8359f8b391ca57bfaf8,2020-05-16T13:11:43.000Z,4001487383951324,940.21,0,1,940.21,23,1225.213043,0.767385,0.767385,0.043478
2,a8830365a7b881167c8e5c9daf717f89,2020-05-16T20:57:35.000Z,4001487383951324,865.45,0,1,865.45,23,1259.573913,0.687097,0.687097,0.043478
3,c6bbd001c206c1214e5aa2fdd12a7e35,2020-05-17T13:51:00.000Z,4001487383951324,89.21,0,1,89.21,23,1259.200000,0.070847,0.070847,0.043478
4,2b2bb2583079a6b8f737b40140d7754a,2020-05-17T16:41:19.000Z,4001487383951324,25.71,0,1,25.71,24,1207.804583,0.021287,0.021287,0.041667


In [5]:
test_df.head()
print(test_df.shape)

(603208, 12)


## Test out the solution, end to end

In [139]:
def get_cloudwatch_logs_url(start_time, end_time):
    log_group_name = '/aws/lambda/' + predict_lambda_name 
    # get the latest log stream for our Lambda that makes fraud predictions
    cw_client = boto3.client('logs')
    last_cw_evt = 0
    while last_cw_evt < int(start_test_time * 1000):
        streams = cw_client.describe_log_streams(logGroupName=log_group_name,
                                                 orderBy='LastEventTime',
                                                 descending=True)['logStreams']
        last_cw_evt = streams[0]['lastIngestionTime'] #'lastEventTimestamp']
        latest_stream = str(streams[0]['logStreamName']).replace('/', '$252F').replace('[$LATEST]', '$255B$2524LATEST$255D')
        if last_cw_evt < int(start_test_time * 1000):
            print('waiting for updated log stream...')
            time.sleep(10)

    # produce a valid URL to get to that log stream
    region = boto3.session.Session().region_name
    log_group_escaped = log_group_name.replace('/', '$252F')
    cw_url = f'https://console.aws.amazon.com/cloudwatch/home?region={region}#logsV2:log-groups/log-group/{log_group_escaped}'
    time_filter = f'$26start$3D{int(start_test_time * 1000) - 10000}$26end$3D{int(end_test_time * 1000) + 40000}'
    full_cw_url = f'{cw_url}/log-events/{latest_stream}$3FfilterPattern$3DPrediction+{time_filter}'
    print('Updated log stream is ready.')
    return full_cw_url

In [30]:
def put_to_stream(stream_name, cc_num, merchant, amount, zip_code, timestamp):
    
    payload = {
        'cc_num': int(cc_num),
        'merchant': merchant,
        'amount': amount,
        'zip_code': zip_code,
        'trans_ts': timestamp
    }
    ret_status = True
    data = json.dumps(payload)
    print(f'Sending transaction on card: {cc_num}...')
    response = kinesis_client.put_record(StreamName = stream_name,
                                             Data = data,
                                             PartitionKey = 'shard1')
    
    if (response['ResponseMetadata']['HTTPStatusCode'] != 200):
        print("ERROR: Kinesis put_record failed: \n{}".format(json.dumps(response)))
        ret_status = False
        
    return ret_status

In [98]:
def simulate_fraud(stream_name, cc_num):
    max_wait = 2
    for i in range(10):
        random_amt = round(np.random.uniform(1.00, 5000.00), 2)
        seconds_to_wait = np.random.uniform(0.5, max_wait)
        print(f'waiting {seconds_to_wait:.1f} seconds to send trans {i}...')
        time.sleep(seconds_to_wait)
        put_to_stream(stream_name, int(cc_num), 'Random Corp', random_amt, '03099', time.time())

### Send some transactions, and see the results
To show that the solution works, we send a single transaction to each of three different credit cards. Then, we simulate a fraud attack on a 4th credit card by sending many quick transactions. The output from our Lambda function is shown below. As expected, the first three one-off transactions are predicted as NOT FRAUD. Of the ten fraudulent transactions, the first is predicted as NOT FRAUD, and the rest are all correctly identified as FRAUD. Notice how the aggregate features are kept current, helping drive more accurate predictions.

In [136]:
cc_nums = test_df.cc_num.unique()[50:54]

start_test_time = time.time() 

put_to_stream(STREAM_NAME, cc_nums[0], 'Merchant-0', round(np.random.uniform(100, 5000), 2), 'zip-0', time.time())
put_to_stream(STREAM_NAME, cc_nums[1], 'Merchant-1', round(np.random.uniform(100, 5000), 2), 'zip-1', time.time())
put_to_stream(STREAM_NAME, cc_nums[2], 'Merchant-2', round(np.random.uniform(100, 5000), 2), 'zip-2', time.time())

print('\nNow simulate a fraud attack...')
fraud_cc_num = cc_nums[3]
simulate_fraud(STREAM_NAME, fraud_cc_num)

end_test_time = time.time() 

Sending transaction on card: 4135583439783239...
Sending transaction on card: 4139276375485267...
Sending transaction on card: 4140512527773862...

Now simulating fraud attack...
waiting 0.7 seconds to send trans 0...
Sending transaction on card: 4147512691589717...
waiting 1.4 seconds to send trans 1...
Sending transaction on card: 4147512691589717...
waiting 1.1 seconds to send trans 2...
Sending transaction on card: 4147512691589717...
waiting 1.1 seconds to send trans 3...
Sending transaction on card: 4147512691589717...
waiting 0.9 seconds to send trans 4...
Sending transaction on card: 4147512691589717...
waiting 1.7 seconds to send trans 5...
Sending transaction on card: 4147512691589717...
waiting 1.2 seconds to send trans 6...
Sending transaction on card: 4147512691589717...
waiting 2.0 seconds to send trans 7...
Sending transaction on card: 4147512691589717...
waiting 1.7 seconds to send trans 8...
Sending transaction on card: 4147512691589717...
waiting 1.6 seconds to send t

### Results can be seen in the CloudWatch log stream of our Lambda function

In [142]:
from IPython.core.display import display, HTML

full_cw_url = get_cloudwatch_logs_url(start_test_time, end_test_time)
display(HTML(f'<b>Review results in this log stream <a target="blank" href="{full_cw_url}">Lambda fraud detection results</a></b>'))

Updated log stream is ready.


### Feed a stream of transactions [optional]
If you would like to send additional credit card transactions to simulate more input traffic to the feature pipeline, you can pull from the test dataset as shown below. Just pass in how many transactions you want to send, and the max wait time between transactions (in seconds).

In [126]:
import time

def simulate_traffic(df, max_wait, num_trans):
    for i in range(num_trans):
        row = test_df.iloc[i]
        cc_num = row['cc_num']
        zip_code = '0'
        merchant = 'A'
        amt = row['amount']
        print(f'cc_num: {cc_num}, amt: {amt}')
        seconds_to_wait = int(np.random.uniform(0.1, max_wait))
        print(f'waiting {seconds_to_wait} seconds to send trans {i}...')
        time.sleep(seconds_to_wait)
        print(f' putting trans with card: {cc_num}, amt: {amt}, zip: {zip_code}, merchant: {merchant}')
        status = put_to_stream(cc_num, merchant, amt, zip_code, now)
        if (not status):
            print('Error found during write to Kinesis Stream')
            break
        i += 1
        if i > (num_trans -1):
            break

In [129]:
#simulate_traffic(test_df, 2, 5)